In [ ]:
!pip install pennylane
!pip install scikit-learn

STEP 1

CHOSSING THE DATASET AND PREPROCESSING

In [9]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
import pennylane as qml
from pennylane import numpy as pnp
from sklearn.svm import SVC
from scipy.optimize import minimize


iris = load_iris()
X = iris.data[iris.target != 2]  # Select only two classes
y = iris.target[iris.target != 2]

scaler = StandardScaler()
X = scaler.fit_transform(X)


sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_index, test_index = next(sss.split(X, y))
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]


print("Training class distribution:", np.bincount(y_train))
print("Test class distribution:", np.bincount(y_test))


if len(np.unique(y_train)) < 2 or len(np.unique(y_test)) < 2:
    raise ValueError("The number of classes has to be greater than one in both training and test sets.")

Training class distribution: [40 40]
Test class distribution: [10 10]


STEP 2

QUANTAM FEATURE MAP

In [10]:
num_qubits = X_train.shape[1]  # Number of qubits should match the number of features

dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def quantum_kernel_circuit(x1, x2):
    for i in range(num_qubits):
        qml.RX(x1[i], wires=i)
        qml.RY(x1[i], wires=i)
    for i in range(num_qubits):
        qml.RX(x2[i], wires=i)
        qml.RY(x2[i], wires=i)
    return qml.state()

def quantum_kernel(params, x1, x2):
    state1 = quantum_kernel_circuit(x1, np.zeros_like(x1))
    state2 = quantum_kernel_circuit(x2, np.zeros_like(x2)).conj()
    return np.abs(np.dot(state1, state2)) ** 2

STEP 3

***Compute QEK Matrix with Progress Tracking***

In [14]:
def quantum_kernel_matrix(X1, X2, params, verbose=False):
    qk_matrix = np.zeros((len(X1), len(X2)))

    total_combinations = len(X1) * len(X2)
    for i, x1 in enumerate(X1):
        for j, x2 in enumerate(X2):
            qk_matrix[i, j] = quantum_kernel(params, x1, x2)

            if verbose:
                print(f"Computing kernel matrix: {i*len(X2)+j+1}/{total_combinations} completed", end="\r")

    if verbose:
        print()
    return qk_matrix

STEP 4

***Optimize Variational Parameters with Fixed Iterations***

In [16]:
def kernel_target_alignment(K, y):
    y = y * 2 - 1  # Convert {0, 1} to {-1, 1}
    Ky = K @ y
    alignment = np.dot(y, Ky) / np.linalg.norm(Ky)
    return alignment

def objective_function(params, iteration=[0]):
    iteration[0] += 1
    if iteration[0] > 10:
        return 0  # End optimization if iteration exceeds 10

    print(f"Optimization iteration {iteration[0]}: Computing kernel matrix...")
    K = quantum_kernel_matrix(X_train, X_train, params, verbose=True)
    alignment = kernel_target_alignment(K, y_train)
    print(f"Optimization iteration {iteration[0]}: Alignment = {-alignment:.4f}")
    return -alignment

initial_params = np.random.rand(num_qubits * 2)  # Initial parameters for RX and RY
opt_result = minimize(objective_function, initial_params, method='COBYLA')
optimized_params = opt_result.x

Optimization iteration 1: Computing kernel matrix...
Computing kernel matrix: 6400/6400 completed
Optimization iteration 1: Alignment = -8.8083
Optimization iteration 2: Computing kernel matrix...
Computing kernel matrix: 6400/6400 completed
Optimization iteration 2: Alignment = -8.8083
Optimization iteration 3: Computing kernel matrix...
Computing kernel matrix: 6400/6400 completed
Optimization iteration 3: Alignment = -8.8083
Optimization iteration 4: Computing kernel matrix...
Computing kernel matrix: 6400/6400 completed
Optimization iteration 4: Alignment = -8.8083
Optimization iteration 5: Computing kernel matrix...
Computing kernel matrix: 6400/6400 completed
Optimization iteration 5: Alignment = -8.8083
Optimization iteration 6: Computing kernel matrix...
Computing kernel matrix: 6400/6400 completed
Optimization iteration 6: Alignment = -8.8083
Optimization iteration 7: Computing kernel matrix...
Computing kernel matrix: 6400/6400 completed
Optimization iteration 7: Alignment = 

STEP 5

***Noise Mitigation and Regularization***

In [24]:
def apply_noise_mitigation_and_regularization(kernel_matrix):

    regularization_term = 1e-3
    num_samples, num_features = kernel_matrix.shape


    regularization_matrix = regularization_term * np.eye(num_samples, num_features)


    kernel_matrix += regularization_matrix

    return kernel_matrix


print("Computing training kernel matrix...")
qk_matrix_train_optimized = quantum_kernel_matrix(X_train, X_train, optimized_params, verbose=True)
qk_matrix_train_optimized = apply_noise_mitigation_and_regularization(qk_matrix_train_optimized)


print("Computing test kernel matrix...")
qk_matrix_test_optimized = quantum_kernel_matrix(X_test, X_train, optimized_params, verbose=True)
qk_matrix_test_optimized = apply_noise_mitigation_and_regularization(qk_matrix_test_optimized)

Computing training kernel matrix...
Computing kernel matrix: 6400/6400 completed
Computing test kernel matrix...
Computing kernel matrix: 1600/1600 completed


step 6

***Train SVM Classifier***

In [25]:
svm = SVC(kernel='precomputed')
svm.fit(qk_matrix_train_optimized, y_train)

SVC(kernel='precomputed')

step 7

*** Evaluate Classifier***

In [26]:
y_pred = svm.predict(qk_matrix_test_optimized)
accuracy = np.mean(y_pred == y_test)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

Classification Accuracy: 100.00%
